In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt
import pyarrow as pa

import os

---

## Taxi Data

In [147]:
import pyarrow.fs
import pyarrow.dataset as ds
#pyarrow.parquet.read_table("s3://ursa-labs-taxi-data/2019/06/data.parquet")

#s3, path = pyarrow.fs.S3FileSystem().from_uri("s3://ursa-labs-taxi-data/2019/")
#dataset = pyarrow.parquet.ParquetDataset(path, filesystem=s3)

dataset = ds.dataset("s3://ursa-labs-taxi-data/2019/")

In [148]:
dataset.schema

vendor_id: string
pickup_at: timestamp[us]
dropoff_at: timestamp[us]
passenger_count: int8
trip_distance: float
rate_code_id: string
store_and_fwd_flag: string
pickup_location_id: int32
dropoff_location_id: int32
payment_type: string
fare_amount: float
extra: float
mta_tax: float
tip_amount: float
tolls_amount: float
improvement_surcharge: float
total_amount: float
congestion_surcharge: float
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 2548

In [149]:
dataset.files

['ursa-labs-taxi-data/2019/01/data.parquet',
 'ursa-labs-taxi-data/2019/02/data.parquet',
 'ursa-labs-taxi-data/2019/03/data.parquet',
 'ursa-labs-taxi-data/2019/04/data.parquet',
 'ursa-labs-taxi-data/2019/05/data.parquet',
 'ursa-labs-taxi-data/2019/06/data.parquet']

In [150]:
import pyarrow.compute as pc

col_sum = 0
count = 0

for batch in dataset.to_batches(columns=["fare_amount"]):
    col_sum += pc.sum(batch.column("fare_amount")).as_py()
    count += batch.num_rows

col_sum / count

13.101317234329155

In [155]:
dataset_local = ds.dataset("/Users/rundel/Scratch/nyc_taxi/",  partitioning=["year", "month"])
dataset_local.files

['/Users/rundel/Scratch/nyc_taxi/2019/01/data.parquet',
 '/Users/rundel/Scratch/nyc_taxi/2019/02/data.parquet',
 '/Users/rundel/Scratch/nyc_taxi/2019/03/data.parquet',
 '/Users/rundel/Scratch/nyc_taxi/2019/04/data.parquet',
 '/Users/rundel/Scratch/nyc_taxi/2019/05/data.parquet',
 '/Users/rundel/Scratch/nyc_taxi/2019/06/data.parquet']

In [156]:
dataset_local.schema

vendor_id: string
pickup_at: timestamp[us]
dropoff_at: timestamp[us]
passenger_count: int8
trip_distance: float
rate_code_id: string
store_and_fwd_flag: string
pickup_location_id: int32
dropoff_location_id: int32
payment_type: string
fare_amount: float
extra: float
mta_tax: float
tip_amount: float
tolls_amount: float
improvement_surcharge: float
total_amount: float
congestion_surcharge: float
year: int32
month: int32
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 2548

In [154]:
col_sum = 0
count = 0

for batch in dataset_local.to_batches(columns=["fare_amount"]):
    col_sum += pc.sum(batch.column("fare_amount")).as_py()
    count += batch.num_rows

col_sum / count

13.101317234329155

In [158]:
dataset_local.to_table(filter = ds.field("fare_amount") > 100).to_pandas()

,vendor_id,pickup_at,dropoff_at,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,year,month
0,2,2019-01-01 00:23:11,2019-01-01 00:23:25,1,0.340000,5,N,265,265,1,160.0,0.0,0.0,25.000000,0.000000,0.3,185.300003,NaN,2019,1
1,2,2019-01-01 00:38:36,2019-01-01 01:21:33,2,33.189999,5,N,107,265,1,121.0,0.0,0.0,0.080000,10.500000,0.3,131.880005,NaN,2019,1
2,1,2019-01-01 00:13:17,2019-01-01 01:06:13,1,44.099998,5,N,132,265,2,150.0,0.0,0.0,0.000000,0.000000,0.3,150.300003,NaN,2019,1
3,2,2018-12-31 23:59:57,2019-01-01 00:00:00,4,0.000000,5,N,264,265,1,121.0,0.0,0.5,36.540001,0.000000,0.3,158.339996,NaN,2019,1
4,2,2019-01-01 00:22:44,2019-01-01 00:22:50,1,0.970000,5,N,265,265,1,120.0,0.0,0.5,30.200001,0.000000,0.3,151.000000,NaN,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34175,2,2019-06-30 23:36:46,2019-07-01 00:26:50,2,36.959999,5,N,132,265,1,158.0,0.0,0.0,34.980000,16.620001,0.3,209.899994,0.0,2019,6
34176,2,2019-06-30 23:21:49,2019-06-30 23:21:52,1,0.000000,5,N,265,265,1,160.0,0.0,0.5,15.000000,0.000000,0.3,175.800003,0.0,2019,6
34177,1,2019-06-30 23:52:45,2019-07-01 00:44:31,1,22.900000,5,N,132,265,1,110.0,0.0,0.0,24.150000,10.500000,0.3,144.949997,0.0,2019,6
34178,1,2019-06-30 23:03:07,2019-07-01 00:02:03,3,47.799999,5,N,138,265,1,225.0,0.0,0.0,0.000000,16.620001,0.3,241.919998,0.0,2019,6


In [159]:
dataset_local.to_table().to_pandas()

,vendor_id,pickup_at,dropoff_at,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,year,month
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.950000,NaN,2019,1
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.299999,NaN,2019,1
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.00,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.800000,NaN,2019,1
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.00,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.550000,NaN,2019,1
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.00,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.549999,NaN,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44459131,1,2019-06-30 23:23:03,2019-06-30 23:39:48,1,0.90,1,N,68,158,1,11.0,3.0,0.5,2.00,0.0,0.3,16.799999,2.5,2019,6
44459132,1,2019-06-30 23:50:22,2019-06-30 23:57:01,1,0.50,1,N,246,90,2,6.0,3.0,0.5,0.00,0.0,0.3,9.800000,2.5,2019,6
44459133,1,2019-06-30 23:58:32,2019-07-01 00:00:42,1,0.20,1,N,90,186,1,3.5,3.0,0.5,1.45,0.0,0.3,8.750000,2.5,2019,6
44459134,2,2019-06-30 23:23:10,2019-06-30 23:30:45,1,1.38,1,N,140,163,1,7.5,0.5,0.5,2.26,0.0,0.3,13.560000,2.5,2019,6
